In [2]:
import pandas as pd
import calendar

neiss_df = pd.read_csv('NEISS2014.csv')
diag_df = pd.read_csv('DiagnosisCodes.csv')

# Add DiagnosisCodes data to NEISS dataframe
neiss_df = pd.merge(neiss_df, diag_df, left_on='diag', right_on='Code')

In [3]:
# Print Nb of patients per weekday
neiss_df['trmt_date'] = pd.to_datetime(neiss_df['trmt_date'], errors='coerce')
neiss_df['weekday'] = neiss_df['trmt_date'].dt.weekday_name

print(neiss_df['weekday'].value_counts())

Saturday     9956
Sunday       9872
Monday       9656
Tuesday      9088
Wednesday    9021
Thursday     9019
Friday       8887
Name: weekday, dtype: int64


In [4]:
neiss_df['month'] = neiss_df['trmt_date'].dt.month
neiss_df['month'] = neiss_df['month'].apply(lambda x: calendar.month_abbr[x])

print(neiss_df['month'].value_counts())

May    6698
Jun    6230
Sep    5865
Mar    5739
Oct    5675
Apr    5584
Jul    5420
Aug    5334
Jan    5076
Feb    4944
Nov    4509
Dec    4425
Name: month, dtype: int64


# Calendar Results

Saturday, Sundays and Mondays have more patients registered than other days of the week.
May and June are the months with the highest numbers of patients while November and December have the least.

---

In [5]:
# Let's explore the fatality data
DISPO_FATILITY_CODE = 8

fatility_df = neiss_df[(neiss_df['disposition'] == DISPO_FATILITY_CODE)]
fatility_df['Diagnosis'].value_counts()

Other/Not Stated                   13
Internal organ injury               7
Contusions, Abrasions               2
Submersion (including Drowning)     2
Fracture                            2
Poisoning                           1
Anoxia                              1
Name: Diagnosis, dtype: int64

In [6]:
# Get % of Other/Not stated deaths
percent_death_other = fatility_df['Diagnosis'].value_counts()[0]/fatility_df['Diagnosis'].value_counts().sum() *100
print("Percentage of 'Other/Not Stated' deaths: %0.1f%%" % percent_death_other)

Percentage of 'Other/Not Stated' deaths: 46.4%


# Fatality Results

The diagnosis that most resulted in deaths was **Other/Not Stated**: 46.4%. 
This shows the need to better record the causes of death.

**Internal organ injury** was the second most common cause of death: 25%


In [7]:
# Get % of Internal organ injury deaths
percent_death_internal = fatility_df['Diagnosis'].value_counts()[1]/fatility_df['Diagnosis'].value_counts().sum() *100
print("Percentage of 'Internal organ injury' deaths: %0.1f%%" % percent_death_internal)

Percentage of 'Internal organ injury' deaths: 25.0%


---


In [8]:
# Let's explore the Transfered to another hospital data
DISPO_TRANSFERED_CODE = 2

transfered_df = neiss_df[(neiss_df['disposition'] == DISPO_TRANSFERED_CODE)]

# Get dataframe that contains total diagnoses
total_diag_df = pd.DataFrame(data=neiss_df['Diagnosis'].value_counts()).reset_index().rename(columns={'index': 'Diagnosis', 'Diagnosis': 'Total'})

transfered_diag_df = pd.DataFrame(data=transfered_df['Diagnosis'].value_counts()).reset_index().rename(columns={'index': 'Diagnosis', 'Diagnosis': 'Transfered Total'}) 

# Create dataframe that contains Hospitalized Number and Total per diagnosis
transfered_diag_df = pd.merge(transfered_diag_df, total_diag_df, on='Diagnosis', how="left")

# Create new column "Transfered Number"/"Total" rate
transfered_diag_df['Rate (%)'] = (transfered_diag_df['Transfered Total']/transfered_diag_df['Total'] * 100)

# Order dataframe by descending by rate
print(transfered_diag_df.sort_values('Rate (%)', ascending=False).reset_index(drop=True).head(10))

                                     Diagnosis  Transfered Total  Total  \
0                                   Amputation                12    125   
1                                   Hemorrhage                 4     64   
2  Burns, thermal (from flames or hot surface)                29    512   
3     Burns, scald (from hot liquids or steam)                20    392   
4                      Ingested foreign object                18    483   
5                     Aspirated foreign object                 2     66   
6                                     Fracture               267   9735   
7                                    Poisoning                18    724   
8                                     Crushing                 3    149   
9                        Internal organ injury               101   5306   

   Rate (%)  
0  9.600000  
1  6.250000  
2  5.664062  
3  5.102041  
4  3.726708  
5  3.030303  
6  2.742681  
7  2.486188  
8  2.013423  
9  1.903505  


# Transfered to other facilities Results

The diagnosis that has the highest transfer rate to other facilities is **Amputation**: 9.6%

Even though **Fractures** is the diagnosis that is the most transfered with **267 cases**, this only represents **2.74% of the total cases**